# TL;DR

**Configuring Enviorment**

installing dependencies

In [42]:
pip install nltk spacy beautifulsoup4 keras tensorflow keras-nlp scikit-learn torch pandas 


Note: you may need to restart the kernel to use updated packages.


In [43]:
import os
import numpy as np
import pandas as pd
from collections import Counter
import nltk
import spacy
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

***Load data***

this data is not from homework 1 and 2 cause i found a better way to extract data from reddit.

In [44]:
dataset = pd.read_csv('data.csv', sep='\t')
dataset.head()

,Unnamed: 0,title,selftext,id,clean_title,clean_selftext,clean_id
0,0,I'm f'ed up and I cannot fix it no matter what...,"When I was younger I was fat, but I'm hilariou...",1e4xvti,im fed up and i cannot fix it no matter what i do,when i was younger i was fat but im hilarious ...,exvti
1,1,I'm 45 years of age and still suck my thumb as...,I'm 45 M and still suck my thumb and honestly ...,1e4xvcj,im years of age and still suck my thumb as a ...,im m and still suck my thumb and honestly don...,exvcj
2,2,"I only paid for half of the price of a cake, k...","More than a year ago, I went to my local groce...",1e4iomq,i only paid for half of the price of a cake kn...,more than a year ago i went to my local grocer...,eiomq
3,3,I lied on my resume because I was desperate fo...,I recently graduated from grad school and have...,1e4fg9e,i lied on my resume because i was desperate fo...,i recently graduated from grad school and have...,efge
4,4,[Extreme Remorse] I relieved my pets from suff...,[Extreme Remorse]\n\nI grew up with pets. Yeah...,1e4cg3w,extreme remorse i relieved my pets from suffering,extreme remorse\n\ni grew up with pets yeah yo...,ecgw


In [45]:
# prompt: select only clean columns and show statistics on  the dataset related to NLP

dataset = dataset[['clean_title', 'clean_selftext','id']]
dataset.dropna(inplace=True)
dataset.info()

# Word count statistics
dataset['title_word_count'] = dataset['clean_title'].apply(lambda x: len(str(x).split()))
dataset['comment_word_count'] = dataset['clean_selftext'].apply(lambda x: len(str(x).split()))
print("Title Word Count Statistics:")
print(dataset['title_word_count'].describe())
print("\nComment Word Count Statistics:")
print(dataset['comment_word_count'].describe())


<class 'pandas.core.frame.DataFrame'>
Index: 822 entries, 0 to 823
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   clean_title     822 non-null    object
 1   clean_selftext  822 non-null    object
 2   id              822 non-null    object
dtypes: object(3)
memory usage: 25.7+ KB
Title Word Count Statistics:
count    822.000000
mean      13.357664
std        3.936284
min        1.000000
25%       11.000000
50%       13.000000
75%       15.000000
max       53.000000
Name: title_word_count, dtype: float64

Comment Word Count Statistics:
count     822.000000
mean      257.379562
std       243.037507
min         3.000000
25%       109.000000
50%       194.500000
75%       326.750000
max      3089.000000
Name: comment_word_count, dtype: float64


In [46]:
# prompt: performing tokenization, as 'processed_comment'
# lemmatization, normalization(stemming and lemmatization), stop-word removal, and word vectorization. after show the modified dataset statistics
nltk.download('punkt')
# Tokenization
dataset['processed_comment'] = dataset['clean_selftext'].apply(nltk.word_tokenize)

# Normalization (Stemming and Lemmatization) and Stop-word Removal
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

def normalize_text(tokens):
    normalized_tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words and word.isalnum()]
    return normalized_tokens

dataset['processed_comment'] = dataset['processed_comment'].apply(normalize_text)

# Word Vectorization (using a simple count vectorizer for demonstration)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([' '.join(tokens) for tokens in dataset['processed_comment']])

# Modified Dataset Statistics
print("Modified Dataset Statistics:")
print(dataset.head())
print("\nShape of word vector matrix:", X.shape)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Modified Dataset Statistics:
                                         clean_title  \
0  im fed up and i cannot fix it no matter what i do   
1  im  years of age and still suck my thumb as a ...   
2  i only paid for half of the price of a cake kn...   
3  i lied on my resume because i was desperate fo...   
4  extreme remorse i relieved my pets from suffering   

                                      clean_selftext       id  \
0  when i was younger i was fat but im hilarious ...  1e4xvti   
1  im  m and still suck my thumb and honestly don...  1e4xvcj   
2  more than a year ago i went to my local grocer...  1e4iomq   
3  i recently graduated from grad school and have...  1e4fg9e   
4  extreme remorse\n\ni grew up with pets yeah yo...  1e4cg3w   

   title_word_count  comment_word_count  \
0                13                  99   
1                14                  99   
2                12                 306   
3                11                 192   
4                 8         

In [47]:
sequences = []
for tokens in dataset['processed_comment']:
  for i in range(1, len(tokens)):
    sequence = tokens[:i]
    sequences.append(sequence)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)
vocab_size = len(tokenizer.word_index) + 1
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

# Split data into input (X) and output (y)
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=vocab_size))

# Build RNN model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length - 1))
model.add(SimpleRNN(16))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model (adjust epochs and batch_size as needed)
model.fit(X, y, epochs=10, verbose=1)

# Example prediction
test_sentence = "this is a test"
test_sequence = tokenizer.texts_to_sequences([test_sentence])[0]
padded_test_sequence = pad_sequences([test_sequence], maxlen=max_length - 1, padding='pre')
prediction = model.predict(padded_test_sequence, verbose=0)
predicted_word_index = np.argmax(prediction)
predicted_word = tokenizer.index_word[predicted_word_index]
print(f"Predicted next word: {predicted_word}")





C:\Users\daniel\PycharmProjects\CrackingtheCodingInterviewquestions\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 438s 142ms/step - accuracy: 0.0152 - loss: 6.8070
Epoch 2/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 456s 148ms/step - accuracy: 0.0178 - loss: 6.2904
Epoch 3/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 448s 146ms/step - accuracy: 0.0253 - loss: 6.2212
Epoch 4/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 448s 146ms/step - accuracy: 0.0322 - loss: 6.1329
Epoch 5/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 451s 147ms/step - accuracy: 0.0425 - loss: 6.0249
Epoch 6/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 456s 149ms/step - accuracy: 0.0519 - loss: 5.9232
Epoch 7/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 460s 150ms/step - accuracy: 0.0572 - loss: 5.8512
Epoch 8/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 464s 151ms/step - accuracy: 0.0604 - loss: 5.8008
Epoch 9/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 462s 151ms/step - accuracy: 0.0611 - loss: 5.7530
Epoch 10/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 449s 146ms/step - accuracy: 0.0627 - loss: 5.7185
Predicted next word: year


In [48]:
#example from class to test
test_data = [
    "I hate programming",
    "NLP is boring",
    "Recurrent neural networks are cool"
]
test_sequences = tokenizer.texts_to_sequences(test_data)
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')
model.predict(test_padded_sequences)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step


array([[1.2814075e-01, 3.4738597e-02, 9.4164386e-03, ..., 1.9855975e-10,
        2.0360254e-10, 2.0174816e-10],
       [1.2814075e-01, 3.4738597e-02, 9.4164386e-03, ..., 1.9855975e-10,
        2.0360254e-10, 2.0174816e-10],
       [1.2814075e-01, 3.4738597e-02, 9.4164386e-03, ..., 1.9855975e-10,
        2.0360254e-10, 2.0174816e-10]], dtype=float32)

In [ ]:

# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, 100, input_length=max_length - 1))
lstm_model.add(LSTM(16))
lstm_model.add(Dense(vocab_size, activation='softmax'))

# Compile the model
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model 
lstm_model.fit(X, y, epochs=10, verbose = 1)

# Evaluate the model
loss, accuracy = lstm_model.evaluate(X, y, verbose=0)
print('LSTM Accuracy: %f' % (accuracy * 100))

Epoch 1/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 573s 186ms/step - accuracy: 0.0150 - loss: 6.8163
Epoch 2/10
3068/3068 ━━━━━━━━━━━━━━━━━━━━ 575s 187ms/step - accuracy: 0.0162 - loss: 6.3051
Epoch 3/10
2026/3068 ━━━━━━━━━━━━━━━━━━━━ 3:23 195ms/step - accuracy: 0.0204 - loss: 6.2613